In [ ]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
# Cargar los descriptores HOG desde el CSV
data = pd.read_csv('/content/drive/MyDrive/hog_train.csv', header=None)

# Separar características (X) y etiquetas (y)
X = data.iloc[:, 1:].values
y = data.iloc[:, 0].values

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear y entrenar el modelo MLP
clf = MLPClassifier(solver='adam',
                    activation='relu',
                    alpha=.001,
                    hidden_layer_sizes=(512, 128, 10),
                    random_state=1,
                    max_iter=500)

clf.fit(X_train, y_train)

# Evaluar el modelo
train_pred = clf.predict(X_train)
train_accuracy = np.mean(train_pred == y_train)

test_pred = clf.predict(X_test)
test_accuracy = np.mean(test_pred == y_test)

print("Training accuracy: {}".format(train_accuracy))
print("Testing accuracy: {}".format(test_accuracy))

Training accuracy: 0.9996875
Testing accuracy: 0.9793333333333333


In [ ]:
import pickle
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# Cargar el modelo guardado de scikit-learn
model_filename = '/content/drive/MyDrive/mlp_model.pkl'
with open(model_filename, 'rb') as file:
    sklearn_model = pickle.load(file)

# Supongamos que las características tienen una forma de (n_samples, n_features)
n_features = sklearn_model.coefs_[0].shape[0]

# Convertir el modelo de scikit-learn a ONNX
initial_type = [('float_input', FloatTensorType([None, n_features]))]
onnx_model = convert_sklearn(sklearn_model, initial_types=initial_type)

# Guardar el modelo ONNX en un archivo
onnx_model_filename = '/content/drive/MyDrive/mlp_model.onnx'
with open(onnx_model_filename, 'wb') as file:
    file.write(onnx_model.SerializeToString())

print("Modelo ONNX guardado en: {}".format(onnx_model_filename))


Modelo ONNX guardado en: /content/drive/MyDrive/mlp_model.onnx


In [ ]:
!pip install tf2onnx


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.4
    Uninstalling protobuf-4.25.4:
      Successfully uninstalled protobuf-4.25.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnxconverter-common 1.14.0 requires protobuf==3.20.2, but you have protobuf 3.20.3 which is incompatible.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.17.0 which is incompatible.


In [ ]:
!python -m tf2onnx.convert --opset 13 --onnx /content/drive/MyDrive/mlp_model.onnx --output /content/drive/MyDrive/mlp_model_tf.pb


/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
usage: convert.py [-h] [--input INPUT] [--graphdef GRAPHDEF] [--saved-model SAVED_MODEL]
                  [--tag TAG] [--signature_def SIGNATURE_DEF]
                  [--concrete_function CONCRETE_FUNCTION] [--checkpoint CHECKPOINT]
                  [--keras KERAS] [--tflite TFLITE] [--tfjs TFJS] [--large_model]
                  [--output OUTPUT] [--inputs INPUTS] [--outputs OUTPUTS]
                  [--ignore_default IGNORE_DEFAULT] [--use_default USE_DEFAULT]
                  [--rename-inputs RENAME_INPUTS] [--rename-outputs RENAME_OUTPUTS]
                  [--use-graph-names] [--opset OPSET] [--dequantize] [--custom-ops CUSTOM_OPS]
                  [--extra_opset EXTRA_OPSET] [--load_op_libraries LOAD_OP_LIBRARIES]
                  [--tar

In [ ]:
import tensorflow as tf

# Cargar el modelo TensorFlow guardado
saved_model_dir = "/content/drive/MyDrive/mlp_model_tf"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Guardar el modelo TensorFlow Lite en un archivo
tflite_model_filename = '/content/drive/MyDrive/mlp_model.tflite'
with open(tflite_model_filename, 'wb') as file:
    file.write(tflite_model)

print("Modelo TensorFlow Lite guardado en: {}".format(tflite_model_filename))


Modelo TensorFlow Lite guardado en: /content/drive/MyDrive/mlp_model.tflite
